# Find tornadoes closest to NEXRAD sites 

In [1]:
import numpy as np
import pandas as pd

## Read all the data
#### tornado file trimmed down from the 1950-2019 file at https://www.spc.noaa.gov/wcm/#data

In [40]:
tornadoes = pd.read_csv('~/Box/MapesGroupUndergradResearch/Tyreek/1991-2019_all_tornadoes.csv')

In [129]:
tornadoes

,om,yr,mo,dy,date,time,tz,st,stf,stn,...,ns,sn,sg,f1,f2,f3,f4,fc,NearestRadar,NearestDist
0,1,1991,1,14,1/14/91,17:00:00,3,TX,48,1,...,1,1,1,287,0,0,0,0,KGRK,74.276494
1,2,1991,1,14,1/14/91,17:50:00,3,TX,48,2,...,1,1,1,21,0,0,0,0,KGRK,88.335741
2,3,1991,1,14,1/14/91,18:14:00,3,TX,48,3,...,1,1,1,21,0,0,0,0,KGRK,84.278825
3,4,1991,1,14,1/14/91,18:18:00,3,TX,48,4,...,1,1,1,89,0,0,0,0,KEWX,142.701384
4,5,1991,1,14,1/14/91,18:18:00,3,TX,48,5,...,1,1,1,481,0,0,0,0,KHGX,100.368893
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36366,618537,2019,12,29,12/29/19,16:03:00,3,MS,28,0,...,1,1,1,7,0,0,0,0,KDGX,110.758644
36367,618538,2019,12,29,12/29/19,16:13:00,3,MS,28,0,...,1,1,1,19,0,0,0,0,KGWX,113.034196
36368,618539,2019,12,29,12/29/19,16:32:00,3,MS,28,0,...,1,1,1,105,0,0,0,0,KGWX,80.398510
36369,618540,2019,12,29,12/29/19,17:13:00,3,MS,28,0,...,1,1,1,101,0,0,0,0,KDGX,81.751804


In [79]:
sites = pd.read_table('~/Box/MapesGroupUndergradResearch/Tyreek/WSR88d_locations.txt')

In [80]:
sites[0:5]

,State,Name,Code,Lat,WestLon
0,PR,San Juan,TJUA,18.115600,66.078064
1,ME,Houlton,KCBW,46.039194,67.806603
2,ME,Gray/Portland,KGYX,43.891355,70.256555
3,VT,Burlington,KCXX,44.510994,73.166424
4,MA,Boston\t,KBOX,41.955892,71.136968


## Loop over tornadoes, calculate all site distances

In [81]:
# Starting longitude: how to get the value
tornadoes[23:24]['slon'].values[0]

-82.47

In [87]:
# Site longitude, how to get the value
sites[11:12]['WestLon'].values[0]

74.4108027

# OK, ready for the magic loop
## For each tornado, calculate distance of START POINT to nearest radar 

Add two columns to tornados DataFrame
1. that distance (so we can sort for the smallest value)
2. radar ID, or better yet build the URL: https://s3.amazonaws.com/noaa-nexrad-level2/index.html#1997/06/06/KAMA/

In [124]:
# Loop and collect Code and Distance lists
NearestCode = []
NearestDist = []

for i in range(len(tornadoes)): 
    # print(tornadoes[i:i+1]['st'].values[0]) # debugging/testing: print the state
    x = tornadoes[i:i+1]['slon'].values[0]
    y = tornadoes[i:i+1]['slat'].values[0]
    min_dist = 1e9 # start with a large value, and find the minimuum
    
    for j in range(len(sites)):
        # print(sites[j:j+1])
        # dx is distance eastward (using cos of latitude correction on sphere)
        dx = (x + sites[j:j+1]['WestLon'].values[0]) * np.cos(y*3.14/180.)
        dy = y - sites[j:j+1]['Lat'].values[0]
        dist2 = dx*dx + dy*dy
        
        if (dist2 < min_dist): 
            # print(sites[j:j+1], '   DISTANCE ', np.sqrt(dist2))
            min_dist = np.sqrt(dist2)
            closest = j 
    #print('') # debugging/testing
    #print('closest station, distance: ')
    #print(sites[closest:closest+1]['Code'].values[0])
    #print('DISTANCE (km): ', min_dist*111.111)
    #print('')
    NearestDist.append(min_dist*111.111) # km 
    NearestCode.append(sites[closest:closest+1]['Code'].values[0])

# Append columns onto the tornadoes DataFrame

In [125]:
tornadoes['NearestRadar'] = NearestCode
tornadoes['NearestDist'] = NearestDist

In [134]:
# build the URL: https://s3.amazonaws.com/noaa-nexrad-level2/index.html#1997/06/06/KAMA/
# Oh too bad the date has to be chopped up and reordered, not trivial

URLs = 'https://s3.amazonaws.com/noaa-nexrad-level2/index.html#' + tornadoes['date'] + '/' + tornadoes['NearestRadar']
URLs[0]

'https://s3.amazonaws.com/noaa-nexrad-level2/index.html#1/14/91/KGRK'

In [128]:
tornadoes.sort_values(by='NearestDist')[0:100]

,om,yr,mo,dy,date,time,tz,st,stf,stn,...,ns,sn,sg,f1,f2,f3,f4,fc,NearestRadar,NearestDist
32645,614915,2017,4,3,4/3/17,12:06:00,3,GA,13,0,...,1,1,1,289,319,0,0,0,KJGX,0.184280
23504,580,2009,5,30,5/30/09,18:48:00,3,OH,39,0,...,1,1,1,27,0,0,0,0,KILN,0.436003
30405,579409,2015,5,16,5/16/15,22:43:00,3,OK,40,0,...,1,1,1,131,0,0,0,0,KINX,0.624151
33569,615813,2017,10,20,10/20/17,3:58:00,3,TX,48,0,...,1,1,1,167,0,0,0,0,KHGX,0.647602
35016,617227,2019,4,13,4/13/19,22:22:00,3,MS,28,0,...,1,1,1,95,0,0,0,0,KGWX,0.834936
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20147,87,2007,3,1,3/1/07,13:48:00,3,AL,1,11,...,2,1,2,45,67,0,0,0,KEOX,4.953299
20148,87,2007,3,1,3/1/07,13:48:00,3,AL,1,11,...,2,0,1,0,0,0,0,0,KEOX,4.953299
13326,446,2001,11,15,11/15/01,18:35:00,3,TX,48,99,...,1,1,1,91,187,0,0,0,KEWX,4.990298
28314,449849,2013,4,11,4/11/13,19:34:00,3,AL,1,0,...,1,1,1,87,81,0,0,0,KMXX,5.011782


# Now sort/screen on some other criteria too: F strengths, ? 